In [1]:
import pandas as pd
import re

# Read the Excel file into a DataFrame
df = pd.read_excel('sample_lg.xlsx')

In [2]:
# Function to convert all text columns in a DataFrame to lowercase
def convert_to_lowercase(df):
    # Create a new DataFrame to store the results
    df_lower = df.copy()
    
    # Loop through each column
    for col in df.columns:
        # Check if the column's data type is object (text)
        if df[col].dtype == 'object':
            # Convert all text in the column to lowercase
            df_lower[col] = df[col].apply(lambda x: str(x).lower() if pd.notna(x) else x)
            
    return df_lower

df = convert_to_lowercase(df)
# Save the modified DataFrame to a new Excel file
df.to_excel('sample_lg_lower.xlsx', sheet_name='LowerCase', index=False)

In [3]:
# Function to rectify line breaks to only occur after a sentence ends
def rectify_line_breaks(text):
    # Replace line breaks followed by a space, or line breaks not followed by a period or a sentence-ending punctuation
    return re.sub(r'(\n\s*|\n(?![.!?]))', ' ', text)

# Apply the function to the 'isi' column
df['isi_rectified'] = df['isi'].apply(lambda x: rectify_line_breaks(str(x)))

# Save the modified DataFrame to a new Excel file
df.to_excel('sample_lg_lower.xlsx', sheet_name='noBreakSentence', index=False)

Mencari list kalimat

In [4]:
# Read the Excel file into a DataFrame from the first sheet (assuming the data is in the first sheet)
df_FW_list = pd.read_excel('list_FW.xlsx')
df_surat = pd.read_excel('sample_lg_lower.xlsx')

In [5]:
# Function to find up to 3 sentences for each word in a list of words, to be applied to 'df_FW_list'
def find_sentences_for_FW(df_text, word):
    found_sentences = []
    sentence_count = 0
    # Loop through each row in the DataFrame containing the text
    for text in df_text['isi_rectified']:
        # Split the text into sentences using regular expressions
        sentences = re.split(r'(?<=[.!?])\s+', str(text))
        for sentence in sentences:
            # Use word boundaries to make sure the word stands alone
            if re.search(r'\b' + re.escape(word.lower()) + r'\b', sentence.lower()):
                sentence_count += 1
                found_sentences.append(sentence)
                if sentence_count >= 1:
                    return found_sentences
    return found_sentences

# Apply the function to each word in 'df_FW_list'
df_FW_list['found_sentences'] = df_FW_list['FW_word'].apply(lambda x: find_sentences_for_FW(df_surat, x))

# Show the first few rows of the DataFrame containing the found sentences for each forbidden word
df_FW_list.head()

,FW_word,found_sentences
0,apply,"[eti’s standard per diem charge shall apply, f..."
1,cycle,[dampak lain apabila kontraktor tidak mengimpl...
2,ass,[drawing pressure products company sight g/ass...
3,upon,"[3 dan 4, dengan ini disampaikan bahwa sejalan..."
4,onal,[menjawab surat kontraktor n0m0r jtb-l- rj-pj-...


In [6]:
# Save the modified DataFrame to a new Excel file
df_FW_list.to_excel('list_sentences_lg.xlsx', sheet_name='sentenceList', index=False)

Membuat list word dalam bentuk notepad

In [7]:
# Function to convert a list of sentences into a list of words
def sentences_to_words(sentences):
    # Split each sentence into words and flatten the list
    words = [word for sentence in sentences for word in sentence.split()]
    return words

# Apply the function to the 'found_sentences' column
df_FW_list['words'] = df_FW_list['found_sentences'].apply(sentences_to_words)

# Write the words to a text file in the format of 'dataset.txt' but without POS tags
output_file_path = 'converted_dataset.txt'
with open(output_file_path, 'w') as f:
    for _, row in df_FW_list.iterrows():
        for word in row['words']:
            f.write(f"{word}\n")

output_file_path


'converted_dataset.txt'

membuat excel list word dari kalimat

In [10]:
df = pd.read_excel('list_sentences_lg.xlsx')

In [12]:
# Function to break down a sentence into its constituent words and symbols
def break_down_sentence_with_symbols(sentence):
    # Use regular expression to tokenize the sentence into words and symbols
    tokens = re.findall(r'\b\w+\b|[.,!?;(){}[\]"\'/\\-]', sentence)
    return tokens

# Create an empty DataFrame to store the broken-down words and symbols
new_df_with_symbols = pd.DataFrame(columns=['Tokens'])

# Loop through each row in the original DataFrame
for index, row in df.iterrows():
    # Get the list of sentences for the current row
    sentences_list = eval(row['found_sentences'])
    
    # Loop through each sentence in the list
    for sentence in sentences_list:
        # Break down the sentence into words and symbols
        tokens = break_down_sentence_with_symbols(sentence)
        
        # Append each token as a new row in the DataFrame
        for token in tokens:
            new_df_with_symbols = new_df_with_symbols.append({'Tokens': token}, ignore_index=True)
        
        # Append a blank row
        new_df_with_symbols = new_df_with_symbols.append({'Tokens': ''}, ignore_index=True)

# Save the new DataFrame to an Excel file
output_file_path_with_symbols = 'brokendown_word.xlsx'
new_df_with_symbols.to_excel(output_file_path_with_symbols, index=False)

new_df_with_symbols.head(10), output_file_path_with_symbols


/var/folders/68/k0y245f57539r_hm0r4dpqtw0000gn/T/ipykernel_63433/1964986379.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_with_symbols = new_df_with_symbols.append({'Tokens': token}, ignore_index=True)
/var/folders/68/k0y245f57539r_hm0r4dpqtw0000gn/T/ipykernel_63433/1964986379.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_with_symbols = new_df_with_symbols.append({'Tokens': token}, ignore_index=True)
/var/folders/68/k0y245f57539r_hm0r4dpqtw0000gn/T/ipykernel_63433/1964986379.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df_with_symbols = new_df_with_symbols.append({'Tokens': token}, ignore_index=True)
/var/folders/68/k0y245f57539r_hm0r4dpqtw0000gn/T/ipykernel_63433/1964986379.py:22: Fu

(     Tokens
 0       eti
 1         s
 2  standard
 3       per
 4      diem
 5    charge
 6     shall
 7     apply
 8         ,
 9       for,
 'brokendown_word.xlsx')